In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurações de visualização
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 16

In [2]:
# Configurar seaborn
sns.set_style("whitegrid")
sns.set_palette("husl")

In [3]:
# Função para carregar e processar os dados
def carregar_dados_b4a(arquivo_path):
    """
    Carrega e processa os dados da aba B4A do arquivo Excel
    """
    # Ler a aba B4A
    df = pd.read_excel(arquivo_path, sheet_name='B4A', header=None)
    
    # Identificar onde começam os dados reais (após o cabeçalho)
    # Geralmente os dados começam após algumas linhas de metadados
    for i, row in df.iterrows():
        if 'TOTAL' in str(row.values[0]):
            data_start = i
            break
    
    # Extrair os nomes das colunas
    col_names = [
        'Categoria',
        'Total_escolas_com_computador',
        'Ate_5_alunos',
        'De_5_1_a_10',
        'De_10_1_a_15',
        'De_15_1_a_20',
        'De_20_1_a_30',
        'De_30_1_a_40',
        'De_40_1_a_50',
        'De_50_1_a_100',
        'Mais_de_100',
        'Sem_computador',
        'Sem_informacao'
    ]
    
    # Criar novo dataframe com os dados processados
    df_clean = df.iloc[data_start:].reset_index(drop=True)
    df_clean.columns = col_names
    
    # Limpar valores não numéricos e converter para números
    for col in col_names[1:]:
        df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    
    return df_clean


In [4]:
# Função principal de análise
def analisar_deficit_tecnologico(arquivo_path):
    """
    Realiza análise completa do déficit tecnológico educacional
    """
    print("=" * 80)
    print("ANÁLISE DO DÉFICIT TECNOLÓGICO EDUCACIONAL BRASILEIRO")
    print("Dados: TIC Educação 2023 - Aba B4A")
    print("=" * 80)
    
    # Carregar dados
    df = carregar_dados_b4a(arquivo_path)

    # 1. ANÁLISE GERAL
    print("\n1. VISÃO GERAL DO ACESSO A COMPUTADORES NAS ESCOLAS")
    print("-" * 50)
    
    total_row = df[df['Categoria'] == 'TOTAL'].iloc[0]
    total_escolas = total_row['Sem_computador'] + total_row['Total_escolas_com_computador']
    
    print(f"Total de escolas analisadas: {total_escolas:,.0f}")
    print(f"Escolas SEM computadores: {total_row['Sem_computador']:,.0f} ({total_row['Sem_computador']/total_escolas*100:.1f}%)")
    print(f"Escolas COM computadores: {total_row['Total_escolas_com_computador']:,.0f} ({total_row['Total_escolas_com_computador']/total_escolas*100:.1f}%)")
    
    # 2. ANÁLISE REGIONAL
    print("\n2. DESIGUALDADE REGIONAL")
    print("-" * 50)
    
    regioes = ['Norte', 'Nordeste', 'Sudeste', 'Sul', 'Centro-Oeste']
    analise_regional = []
    
    for regiao in regioes:
        row = df[df['Categoria'] == regiao].iloc[0]
        total_regiao = row['Sem_computador'] + row['Total_escolas_com_computador']
        pct_sem = (row['Sem_computador'] / total_regiao) * 100
        analise_regional.append({
            'Região': regiao,
            'Total Escolas': total_regiao,
            'Sem Computador': row['Sem_computador'],
            '% Sem Computador': pct_sem,
            'Com Computador': row['Total_escolas_com_computador'],
            '% Com Computador': 100 - pct_sem
        })
    
    df_regional = pd.DataFrame(analise_regional)
    df_regional = df_regional.sort_values('% Sem Computador', ascending=False)
    
    print("\nRanking de Exclusão Digital por Região:")
    for idx, row in df_regional.iterrows():
        print(f"{row['Região']:15} - {row['% Sem Computador']:5.1f}% sem computadores ({row['Sem Computador']:,.0f} de {row['Total Escolas']:,.0f} escolas)")
    
    # 3. ANÁLISE URBANO VS RURAL
    print("\n3. DISPARIDADE URBANO-RURAL")
    print("-" * 50)
    
    for area in ['Urbana', 'Rural']:
        row = df[df['Categoria'] == area].iloc[0]
        total_area = row['Sem_computador'] + row['Total_escolas_com_computador']
        pct_sem = (row['Sem_computador'] / total_area) * 100
        print(f"Área {area:7} - {pct_sem:5.1f}% sem computadores ({row['Sem_computador']:,.0f} de {total_area:,.0f} escolas)")
    
    # 4. ANÁLISE POR DEPENDÊNCIA ADMINISTRATIVA
    print("\n4. SITUAÇÃO POR REDE DE ENSINO")
    print("-" * 50)
    
    deps = ['Municipal', 'Estadual', 'Particular']
    for dep in deps:
        row = df[df['Categoria'] == dep].iloc[0]
        total_dep = row['Sem_computador'] + row['Total_escolas_com_computador']
        pct_sem = (row['Sem_computador'] / total_dep) * 100
        print(f"Rede {dep:10} - {pct_sem:5.1f}% sem computadores ({row['Sem_computador']:,.0f} de {total_dep:,.0f} escolas)")
    
    # 5. ANÁLISE DA PROPORÇÃO ALUNO/COMPUTADOR
    print("\n5. PROPORÇÃO ALUNO/COMPUTADOR NAS ESCOLAS COM EQUIPAMENTOS")
    print("-" * 50)
    
    # Calcular distribuição das escolas com computador por faixa de alunos/computador
    faixas_cols = ['Ate_5_alunos', 'De_5_1_a_10', 'De_10_1_a_15', 'De_15_1_a_20', 
                   'De_20_1_a_30', 'De_30_1_a_40', 'De_40_1_a_50', 'De_50_1_a_100', 'Mais_de_100']
    
    total_row = df[df['Categoria'] == 'TOTAL'].iloc[0]
    total_com_pc = total_row['Total_escolas_com_computador']
    
    print("\nDistribuição das escolas COM computador por proporção aluno/equipamento:")
    for col in faixas_cols:
        valor = total_row[col]
        pct = (valor / total_com_pc) * 100
        faixa_nome = col.replace('_', ' ').replace('De ', '').replace('Ate', 'Até').replace('alunos', 'alunos/PC')
        print(f"  {faixa_nome:25} - {valor:8,.0f} escolas ({pct:5.1f}%)")
    
    # Calcular quantas escolas têm proporção inadequada (>20 alunos/computador)
    inadequadas = sum([total_row[col] for col in ['De_20_1_a_30', 'De_30_1_a_40', 
                                                   'De_40_1_a_50', 'De_50_1_a_100', 'Mais_de_100']])
    pct_inadequadas = (inadequadas / total_com_pc) * 100
    
    print(f"\n⚠️  {inadequadas:,.0f} escolas ({pct_inadequadas:.1f}%) têm mais de 20 alunos por computador")
    print("   (proporção considerada inadequada para uso pedagógico efetivo)")
    
    # 6. IMPLICAÇÕES PARA IA GENERATIVA
    print("\n6. IMPLICAÇÕES PARA O USO DE IA GENERATIVA")
    print("-" * 50)
    
    # Estimar população estudantil afetada (considerando média de alunos por escola)
    media_alunos_escola = 300  # Estimativa conservadora
    
    escolas_sem_pc = total_row['Sem_computador']
    escolas_pc_inadequado = inadequadas
    
    alunos_sem_acesso = escolas_sem_pc * media_alunos_escola
    alunos_acesso_limitado = escolas_pc_inadequado * media_alunos_escola
    
    print(f"\nEstimativa de estudantes afetados:")
    print(f"  • Sem NENHUM acesso a computadores: ~{alunos_sem_acesso/1_000_000:.1f} milhões")
    print(f"  • Com acesso MUITO LIMITADO (>20 alunos/PC): ~{alunos_acesso_limitado/1_000_000:.1f} milhões")
    print(f"  • TOTAL com acesso inadequado para IA: ~{(alunos_sem_acesso + alunos_acesso_limitado)/1_000_000:.1f} milhões")
    
    print("\n🔴 Impactos para uso de IA Generativa:")
    print("  • Impossibilidade de usar ferramentas avançadas (GitHub Copilot, ChatGPT Plus, Claude)")
    print("  • Limitação ao uso mobile básico (consultas simples, não projetos complexos)")
    print("  • Exclusão de atividades de programação, análise de dados e automação")
    print("  • Perpetuação do papel de consumidor vs. criador de tecnologia")
    
    return df, df_regional


In [ ]:
# Função para criar visualizações
def criar_visualizacoes(df, df_regional):
    """
    Cria visualizações dos principais insights
    """
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Déficit Tecnológico Educacional Brasileiro - TIC Educação 2023', fontsize=16, y=1.02)
    
    # 1. Gráfico de barras - Situação por região
    ax1 = axes[0, 0]
    df_regional_plot = df_regional.sort_values('% Sem Computador')
    colors = ['#e74c3c' if x > 60 else '#f39c12' if x > 40 else '#27ae60' 
              for x in df_regional_plot['% Sem Computador']]
    bars = ax1.barh(df_regional_plot['Região'], df_regional_plot['% Sem Computador'], color=colors)
    ax1.set_xlabel('% de Escolas SEM Computadores')
    ax1.set_title('Exclusão Digital por Região')
    ax1.set_xlim(0, 100)
    for bar, val in zip(bars, df_regional_plot['% Sem Computador']):
        ax1.text(val + 1, bar.get_y() + bar.get_height()/2, f'{val:.1f}%', 
                va='center', fontweight='bold')
    
    # 2. Comparação Urbano vs Rural
    ax2 = axes[0, 1]
    areas_data = []
    for area in ['Urbana', 'Rural']:
        row = df[df['Categoria'] == area].iloc[0]
        total = row['Sem_computador'] + row['Total_escolas_com_computador']
        areas_data.append({
            'Área': area,
            'Com Computador': (row['Total_escolas_com_computador']/total)*100,
            'Sem Computador': (row['Sem_computador']/total)*100
        })
    
    df_areas = pd.DataFrame(areas_data)
    x = np.arange(len(df_areas))
    width = 0.35
    
    ax2.bar(x - width/2, df_areas['Com Computador'], width, label='Com Computador', color='#27ae60')
    ax2.bar(x + width/2, df_areas['Sem Computador'], width, label='Sem Computador', color='#e74c3c')
    ax2.set_ylabel('Percentual de Escolas (%)')
    ax2.set_title('Disparidade de Acesso: Urbano vs Rural')
    ax2.set_xticks(x)
    ax2.set_xticklabels(df_areas['Área'])
    ax2.legend()
    ax2.set_ylim(0, 100)
    
    # 3. Rede de ensino
    ax3 = axes[1, 0]
    redes_data = []
    for rede in ['Municipal', 'Estadual', 'Particular']:
        row = df[df['Categoria'] == rede].iloc[0]
        total = row['Sem_computador'] + row['Total_escolas_com_computador']
        pct_sem = (row['Sem_computador'] / total) * 100
        redes_data.append({'Rede': rede, '% Sem Computador': pct_sem})
    
    df_redes = pd.DataFrame(redes_data)
    colors_rede = ['#e74c3c', '#f39c12', '#3498db']
    ax3.bar(df_redes['Rede'], df_redes['% Sem Computador'], color=colors_rede)
    ax3.set_ylabel('% de Escolas SEM Computadores')
    ax3.set_title('Exclusão por Dependência Administrativa')
    ax3.set_ylim(0, 100)
    for i, val in enumerate(df_redes['% Sem Computador']):
        ax3.text(i, val + 2, f'{val:.1f}%', ha='center', fontweight='bold')
    
    # 4. Proporção aluno/computador (escolas que têm computador)
    ax4 = axes[1, 1]
    total_row = df[df['Categoria'] == 'TOTAL'].iloc[0]
    
    categorias = ['Adequada\n(≤20 alunos/PC)', 'Inadequada\n(>20 alunos/PC)', 'Sem Computador']
    
    adequadas = sum([total_row[col] for col in ['Ate_5_alunos', 'De_5_1_a_10', 
                                                 'De_10_1_a_15', 'De_15_1_a_20']])
    inadequadas = sum([total_row[col] for col in ['De_20_1_a_30', 'De_30_1_a_40', 
                                                   'De_40_1_a_50', 'De_50_1_a_100', 'Mais_de_100']])
    sem_pc = total_row['Sem_computador']
    
    valores = [adequadas, inadequadas, sem_pc]
    colors_prop = ['#27ae60', '#f39c12', '#e74c3c']
    
    wedges, texts, autotexts = ax4.pie(valores, labels=categorias, colors=colors_prop, 
                                        autopct='%1.1f%%', startangle=90)
    ax4.set_title('Distribuição das Escolas por Adequação Tecnológica')
    
    # Ajustar layout
    plt.tight_layout()
    
    return fig




In [5]:
# Função para gerar relatório de insights
def gerar_relatorio_ia(df):
    """
    Gera relatório específico sobre implicações para IA Generativa
    """
    print("\n" + "=" * 80)
    print("RELATÓRIO: IMPLICAÇÕES PARA A REVOLUÇÃO DA IA GENERATIVA NO BRASIL")
    print("=" * 80)
    
    print("\n📊 PRINCIPAIS FINDINGS:")
    print("-" * 50)
    
    total_row = df[df['Categoria'] == 'TOTAL'].iloc[0]
    total_escolas = total_row['Sem_computador'] + total_row['Total_escolas_com_computador']
    
    # Finding 1
    print("\n1. EXCLUSÃO MASSIVA:")
    print(f"   • {(total_row['Sem_computador']/total_escolas)*100:.1f}% das escolas brasileiras não têm")
    print("     NENHUM computador disponível para alunos")
    
    # Finding 2
    print("\n2. DESIGUALDADE REGIONAL CRÍTICA:")
    norte_row = df[df['Categoria'] == 'Norte'].iloc[0]
    nordeste_row = df[df['Categoria'] == 'Nordeste'].iloc[0]
    total_norte = norte_row['Sem_computador'] + norte_row['Total_escolas_com_computador']
    total_nordeste = nordeste_row['Sem_computador'] + nordeste_row['Total_escolas_com_computador']
    
    print(f"   • Norte: {(norte_row['Sem_computador']/total_norte)*100:.1f}% sem computadores")
    print(f"   • Nordeste: {(nordeste_row['Sem_computador']/total_nordeste)*100:.1f}% sem computadores")
    print("   • Estas regiões ficarão ainda mais para trás na economia da IA")
    
    # Finding 3
    print("\n3. APARTHEID DIGITAL RURAL:")
    rural_row = df[df['Categoria'] == 'Rural'].iloc[0]
    total_rural = rural_row['Sem_computador'] + rural_row['Total_escolas_com_computador']
    print(f"   • {(rural_row['Sem_computador']/total_rural)*100:.1f}% das escolas rurais sem computadores")
    print("   • População rural será excluída das oportunidades da IA")
    
    # Finding 4
    print("\n4. ELITE TECNOLÓGICA:")
    particular_row = df[df['Categoria'] == 'Particular'].iloc[0]
    total_particular = particular_row['Sem_computador'] + particular_row['Total_escolas_com_computador']
    print(f"   • Escolas particulares: apenas {(particular_row['Sem_computador']/total_particular)*100:.1f}% sem computadores")
    print("   • Criação de uma elite capaz de usar IA vs. massa excluída")
    
    print("\n🚨 CONSEQUÊNCIAS PARA O BRASIL NA ERA DA IA:")
    print("-" * 50)
    print("""
    1. PERPETUAÇÃO DO SUBDESENVOLVIMENTO TECNOLÓGICO
       → Enquanto países desenvolvidos usam IA para automatizar e inovar,
         o Brasil forma uma geração que mal sabe usar um computador
    
    2. PERDA DE COMPETITIVIDADE ECONÔMICA
       → Empresas brasileiras não terão mão de obra capaz de implementar
         soluções de IA, dependendo de consultoria estrangeira
    
    3. AMPLIAÇÃO DA DESIGUALDADE SOCIAL
       → Pequena elite com acesso a computadores dominará empregos de IA
       → Maioria ficará em subempregos ameaçados pela automação
    
    4. EXCLUSÃO DA ECONOMIA DIGITAL GLOBAL
       → Impossibilidade de participar do desenvolvimento de IA
       → Brasil como mero consumidor, não produtor de tecnologia
    
    5. IMPACTO GERACIONAL IRREVERSÍVEL
       → Jovens atuais perderão a janela crítica de aprendizado
       → Gap tecnológico pode levar décadas para ser superado
    """)
    
    print("\n💡 RECOMENDAÇÕES URGENTES:")
    print("-" * 50)
    print("""
    1. PROGRAMA EMERGENCIAL DE COMPUTADORES NAS ESCOLAS
       • Meta: 1 computador para cada 5 alunos em 5 anos
       • Prioridade: Norte e Nordeste + escolas rurais
    
    2. FORMAÇÃO MASSIVA DE PROFESSORES EM IA
       • Capacitar docentes para ensinar com e sobre IA
       • Integrar IA no currículo desde o ensino fundamental
    
    3. PARCERIAS PÚBLICO-PRIVADAS
       • Big techs brasileiras devem adotar escolas
       • Criar laboratórios de IA em escolas públicas
    
    4. POLÍTICA NACIONAL DE IA NA EDUCAÇÃO
       • Criar diretrizes curriculares para ensino de IA
       • Estabelecer metas de alfabetização em IA
    """)


In [7]:
# Script principal
if __name__ == "__main__":
    # Caminho do arquivo
    arquivo = "tic_educacao_2023_escolas_tabela_total_v1.0.xlsx"
    
    try:
        # Executar análise
        df, df_regional = analisar_deficit_tecnologico(arquivo)
        
        # Gerar relatório sobre IA
        gerar_relatorio_ia(df)
        
        # Criar visualizações
        fig = criar_visualizacoes(df, df_regional)
        plt.savefig('deficit_tecnologico_educacional.png', dpi=300, bbox_inches='tight')
        plt.show()
        
        # Exportar dados processados
        df.to_csv('dados_processados_b4a.csv', index=False)
        df_regional.to_csv('analise_regional.csv', index=False)
        
        print("\n✅ Análise concluída com sucesso!")
        print("📁 Arquivos gerados:")
        print("   • deficit_tecnologico_educacional.png")
        print("   • dados_processados_b4a.csv")
        print("   • analise_regional.csv")
        
    except FileNotFoundError:
        print(f"⚠️  Arquivo '{arquivo}' não encontrado!")
        print("Por favor, certifique-se de que o arquivo está no diretório correto.")
    except Exception as e:
        print(f"❌ Erro durante a análise: {str(e)}")
        print("Verifique se o arquivo possui a aba 'B4A' com a estrutura esperada.")

ANÁLISE DO DÉFICIT TECNOLÓGICO EDUCACIONAL BRASILEIRO
Dados: TIC Educação 2023 - Aba B4A

1. VISÃO GERAL DO ACESSO A COMPUTADORES NAS ESCOLAS
--------------------------------------------------
Total de escolas analisadas: nan
Escolas SEM computadores: 63,459 (nan%)
Escolas COM computadores: nan (nan%)

2. DESIGUALDADE REGIONAL
--------------------------------------------------
❌ Erro durante a análise: single positional indexer is out-of-bounds
Verifique se o arquivo possui a aba 'B4A' com a estrutura esperada.
